In [1]:
import caffe
from dataLoader import fc7FrameSequenceGenerator
from hdf5_npstreamsequence_generator import HDF5SequenceWriter
from fileWriter import *
from utils import *

In [23]:
MAX_WORDS = 100
BUFFER_SIZE = 1 # TEXT streams
NUM_OUT_PER_CHUNK = 20
START_CHUNK = 0

SETTING = '/data/gengshan/pose_s2vt'
FRAMEFEAT_FILE_PATTERN = SETTING + '/splits/dataCsv_{0}.txt'
SENTS_FILE_PATTERN = SETTING + '/splits/dataTsv_{0}.txt'  # input paths
vocab_file = '/data/gengshan/s2vt/vocabulary_whole.txt'

LSTM_NET_FILE = './s2vt.words_to_preds.deploy.prototxt'
RESULTS_DIR = SETTING + '/results'
NET_TAG = 's2vt_asl_pose_iter_11668'
# NET_TAG = 's2vt_asl_pose_iter_40809'
MODEL_FILE = SETTING + '/snapshots/stored/' + NET_TAG +'.caffemodel'
# MODEL_FILE = SETTING + '/snapshots/' + NET_TAG +'.caffemodel'
STRATEGIES = [{'type': 'beam', 'beam_size': 1}]

data_split_name = 'haha'   # sv files

filenames =  [FRAMEFEAT_FILE_PATTERN.format(data_split_name),
               SENTS_FILE_PATTERN.format(data_split_name)]

In [24]:
fsg = fc7FrameSequenceGenerator(filenames, BUFFER_SIZE,
      vocab_file, max_words=MAX_WORDS, align=False, shuffle=False,
      pad=False, truncate=False, reverse = False)

Reading frame features from file: /data/gengshan/pose_s2vt/splits/dataCsv_haha.txt
Reading sentences in: /data/gengshan/pose_s2vt/splits/dataTsv_haha.txt
Initializing the vocabulary.
Initialized vocabulary from file with 6067 unique words (from 6066 total words in dataset).


In [25]:
video_gt_pairs = all_video_gt_pairs(fsg)  # target sentence
print 'Read %d videos pool feats' % len(fsg.vid_framefeats)
NUM_CHUNKS = 2  # (len(fsg.vid_framefeats)/NUM_OUT_PER_CHUNK) + 1
# add english inverted vocab 
vocab_list = ['<EOS>'] + fsg.vocabulary_inverted

Processed 0/600 (0.000000%) lines
Processed 1/600 (0.166667%) lines
Processed 100/600 (16.666667%) lines
Processed 200/600 (33.333333%) lines
Processed 300/600 (50.000000%) lines
Processed 400/600 (66.666667%) lines
Processed 500/600 (83.333333%) lines
Processed 0/600 (0.000000%) lines
Read 600 videos pool feats


In [26]:
caffe.set_mode_gpu()
caffe.set_device(0)
lstm_net = caffe.Net(LSTM_NET_FILE, MODEL_FILE, caffe.TEST)

In [27]:
for c in range(START_CHUNK, NUM_CHUNKS):
    chunk_start = c * NUM_OUT_PER_CHUNK
    chunk_end = (c + 1) * NUM_OUT_PER_CHUNK
    chunk = video_gt_pairs.keys()[chunk_start:chunk_end]
    html_out_filename = '%s/%s.%s.%d_to_%d.html' % \
      (RESULTS_DIR, data_split_name, NET_TAG, chunk_start, chunk_end)
    text_out_filename = '%s/%s.%s_' % \
      (RESULTS_DIR, data_split_name, NET_TAG)

    if not os.path.exists(RESULTS_DIR): os.makedirs(RESULTS_DIR)
    outputs = run_pred_iters(lstm_net, chunk, video_gt_pairs,
                fsg, strategies=STRATEGIES, display_vocab=vocab_list)

    html_out = to_html_output(outputs, vocab_list)
    html_out_file = open(html_out_filename, 'w')
    html_out_file.write(html_out)
    html_out_file.close() 
    # break
            

Num video features: 130 
Dimension of video features: (1, 32)
[[1, 1, 1, 3516, 1, 34, 1101, 1080, 17, 1, 1025, 17, 2, 1, 0]]
[{'caption': [1, 1, 1, 3516, 1, 34, 1101, 1080, 17, 1, 1025, 17, 2, 1, 0], 'gt': False, 'prob': [0.98784596, 0.95067114, 0.82986957, 0.43372169, 0.99899989, 0.95077318, 0.9897517, 0.97961217, 0.98964149, 0.99930656, 0.97247314, 0.9998883, 0.99997413, 0.99927467, 0.99992502], 'source': {'beam_size': 1, 'type': 'beam'}}]
Generated caption (length 15, log_p = -1.206992, log_p_word = -0.080466):
<en_unk> <en_unk> <en_unk> carefully <en_unk> its historical relationship with <en_unk> starting with the <en_unk>.
Num video features: 37 
Dimension of video features: (1, 32)
[[1, 45, 1, 2, 1, 1, 12, 1, 0]]
[{'caption': [1, 45, 1, 2, 1, 1, 12, 1, 0], 'gt': False, 'prob': [0.99972194, 0.96269906, 0.99856406, 0.99486876, 0.99992383, 0.99959618, 0.99835509, 0.99958581, 0.99946517], 'source': {'beam_size': 1, 'type': 'beam'}}]
Generated caption (length 9, log_p = -0.047950, log

In [6]:
vocab_inds_to_sentence(vocab_list, outputs['--u_-w9WCG8_14'][0]['caption'])

KeyError: '--u_-w9WCG8_14'

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 4,
 563,
 113,
 275,
 558,
 360,
 0]

In [39]:
for it, frame in enumerate(x):
    if it == 0:
        ifSeq = 0
    else:
        ifSeq = 1
    lstm_net.forward(frames_fc7=frame, cont_sentence=np.array([ifSeq]), input_sentence=np.array([0]), stage_indicator=np.array([0]))  # 0 is EOS, 1 is unique token

In [29]:
x = video_to_descriptor('FvWTyzjD690_235', fsg)

In [27]:
x[x > 0] = 0

In [28]:
x

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.]])

In [80]:
lstm_net.blobs['cont_sentence'].data.shape

(100, 1)

[-1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 15.0,
 2.0,
 151.0,
 1.0,
 0.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0]

In [25]:
video_gt_pairs['FvWTyzjD690_235'].shape

KeyError: 'FvWTyzjD690_235'

In [18]:
import h5py
data = h5py.File('/data/gengshan/pose_s2vt/hdf5/buffer_32_s2vt_100/test_batches/batch_0.h5', 'r')

In [34]:
list(data['cont_sentence'][:100, 0])

[0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [39]:
vocab_inds_to_sentence(vocab_list, [int(x) for x in data['target_sentence'][:100,0]])

'01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 have the same <en_unk> <EOS> 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01...'

In [88]:
probs = lstm_net.forward(frames_fc7=data['frame_fc7'][:100, :1, :], \
                 cont_sentence=data['cont_sentence'][:100, :1],\
                 input_sentence=data['input_sentence'][:100, :1],\
                 stage_indicator=data['stage_indicator'][:100, :1])['probs']

In [89]:
probs.shape

(100, 1, 6068)

In [17]:
lstm_net